In [ ]:
# Setup + Load Data
import pandas as pd
import numpy as np
from google.colab import files

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

In [ ]:
# Upload file from computer
df = pd.read_csv("/content/Access_to_Tech_Dataset.csv")
df.head()

,id,web_URL_id,domain_category,web_URL,scrape_status,html_file_name,html_file_path,violation_count,violation_name,violation_score,violation_description,violation_description_url,affected_html_elements,violation_category,violation_impact,wcag_reference,supplementary_information
0,700_0,700,Government and Public Services,https://www.usa.gov/about-the-us,scraped,www_usa_gov_about_the_us.html,/content/workspace/FullPipeline/html_pages_asy...,4,color-contrast-enhanced,4,Ensures the contrast between foreground and ba...,https://dequeuniversity.com/rules/axe/4.4/colo...,"<a href=\/buy-from-government\"">How to buy fro...",Layout,serious,['1.4.6 Contrast (Enhanced)'],"{'fgColor': '#00bde3', 'bgColor': '#112f4e', '..."
1,700_1,700,Government and Public Services,https://www.usa.gov/about-the-us,scraped,www_usa_gov_about_the_us.html,/content/workspace/FullPipeline/html_pages_asy...,4,landmark-banner-is-top-level,3,Ensures the banner landmark is at top level,https://dequeuniversity.com/rules/axe/4.4/land...,"<div class=\usa-banner__header\"" role=\""banner...",Syntax,moderate,['1.3.1 Info and Relationships'],"<div class=\usa-banner__header\"" role=\""banner..."
2,700_2,700,Government and Public Services,https://www.usa.gov/about-the-us,scraped,www_usa_gov_about_the_us.html,/content/workspace/FullPipeline/html_pages_asy...,4,landmark-no-duplicate-banner,3,Ensures the document has at most one banner la...,https://dequeuniversity.com/rules/axe/4.4/land...,<header class=\usa-header usa-header--extended...,Syntax,moderate,['1.3.1 Info and Relationships'],11 <header> or role='banner' elements found:\n...
3,700_3,700,Government and Public Services,https://www.usa.gov/about-the-us,scraped,www_usa_gov_about_the_us.html,/content/workspace/FullPipeline/html_pages_asy...,4,landmark-unique,3,Landmarks should have a unique role or role/la...,https://dequeuniversity.com/rules/axe/4.4/land...,<header class=\usa-header usa-header--extended...,Syntax,moderate,['1.3.1 Info and Relationships'],Role 'search' found 2 times:\n<form accept-cha...
4,701_0,701,Government and Public Services,https://www.usa.gov/benefits,scraped,www_usa_gov_benefits.html,/content/workspace/FullPipeline/html_pages_asy...,4,color-contrast-enhanced,4,Ensures the contrast between foreground and ba...,https://dequeuniversity.com/rules/axe/4.4/colo...,"<a href=\/food-help\"">Learn about food assista...",Layout,serious,['1.4.6 Contrast (Enhanced)'],"{'fgColor': '#00bde3', 'bgColor': '#112f4e', '..."


In [ ]:
# Cleaning: filter scrapes + standardize key fields
df_clean = df.copy()

# Normalize scrape_status to lowercase strings
if "scrape_status" in df_clean.columns:
    df_clean["scrape_status_norm"] = df_clean["scrape_status"].astype(str).str.strip().str.lower()
else:
    df_clean["scrape_status_norm"] = "unknown"

# Keep only successful scrapes
success_values = {"success", "successful", "ok", "true", "1", "yes", "scraped"}
if "scrape_status" in df_clean.columns:
    before = len(df_clean)
    df_clean = df_clean[df_clean["scrape_status_norm"].isin(success_values)].copy()
    after = len(df_clean)
    print(f"Filtered successful scrapes: {before} -> {after}")
else:
    print("No scrape_status column found; skipping scrape filter.")

Filtered successful scrapes: 3524 -> 3524


In [ ]:
# Normalize text fields
def normalize_text(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

for col in ["domain_category", "violation_name", "violation_category", "violation_impact", "wcag_reference"]:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()

# Make consistent domain labels (lowercase)
if "domain_category" in df_clean.columns:
    df_clean["domain_category"] = normalize_text(df_clean["domain_category"])

# Make consistent violation_category labels (lowercase)
if "violation_category" in df_clean.columns:
    df_clean["violation_category"] = normalize_text(df_clean["violation_category"])

In [ ]:
# Numeric cleaning
# Convert score/count fields to numeric where possible
for col in ["violation_score", "violation_count"]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

# If violation_count is missing per-row, set to 1 (since each record is a violation instance)
# BUT only do this if the column exists and has many NaNs.
if "violation_count" in df_clean.columns:
    nan_rate = df_clean["violation_count"].isna().mean()
    print("violation_count NaN rate:", round(nan_rate, 3))
    if nan_rate > 0.2:
        df_clean["violation_count"] = df_clean["violation_count"].fillna(1)
else:
    # Create a count column for aggregation
    df_clean["violation_count"] = 1

violation_count NaN rate: 0.0


In [ ]:
# Website-level aggregation table (for clustering + domain analysis)
# Make website key
website_key = None
for candidate in ["web_URL_id", "web_URL"]:
    if candidate in df_clean.columns:
        website_key = candidate
        break

if website_key is None:
    raise ValueError("No website identifier found (expected web_URL_id or web_URL).")

print("Using website key:", website_key)

Using website key: web_URL_id


In [ ]:
# Website-level numeric aggregates
group_cols = [website_key]
if "domain_category" in df_clean.columns:
    # Domain should be constant per website in ideal case; keep first
    pass

agg_dict = {
    "violation_count": "sum",
}

if "violation_score" in df_clean.columns:
    agg_dict["violation_score"] = ["mean", "median", "max"]

website_agg = df_clean.groupby(group_cols).agg(agg_dict)
website_agg.columns = ["_".join([c for c in col if c]).strip("_") for col in website_agg.columns.to_flat_index()]
website_agg = website_agg.reset_index()

# Add domain category if present
if "domain_category" in df_clean.columns:
    domain_map = df_clean.groupby(website_key)["domain_category"].agg(lambda x: x.mode().iloc[0] if len(x.mode()) else x.iloc[0]).reset_index()
    website_agg = website_agg.merge(domain_map, on=website_key, how="left")

display(website_agg.head())
print("Website-level table shape:", website_agg.shape)


,web_URL_id,violation_count_sum,violation_score_mean,violation_score_median,violation_score_max,domain_category
0,16,1,4.000000,4.0,4,government and public services
1,17,16,3.250000,3.0,4,government and public services
2,20,9,3.000000,3.0,4,government and public services
3,21,49,3.571429,4.0,5,news and media
4,26,49,4.000000,4.0,5,news and media


Website-level table shape: (591, 6)


In [ ]:
# Website × violation_category pivot (for clustering + stacked bars)
# Pivot counts by violation_category
if "violation_category" in df_clean.columns:
    cat_pivot = pd.pivot_table(
        df_clean,
        index=website_key,
        columns="violation_category",
        values="violation_count",
        aggfunc="sum",
        fill_value=0
    ).reset_index()
else:
    cat_pivot = pd.DataFrame({website_key: df_clean[website_key].unique()})

display(cat_pivot.head())
print("Category pivot shape:", cat_pivot.shape)

violation_category,web_URL_id,layout,nan,semantic,syntax
0,16,1,0,0,0
1,17,4,0,0,12
2,20,3,0,0,6
3,21,7,0,0,42
4,26,14,0,0,35


Category pivot shape: (591, 5)


In [ ]:
# Website × top violation_name pivot (for better clustering)
# Pivot counts by top violation_name
TOP_N = 25

if "violation_name" in df_clean.columns:
    top_names = (
        df_clean.groupby("violation_name")["violation_count"].sum()
        .sort_values(ascending=False)
        .head(TOP_N)
        .index
    )
    df_top = df_clean[df_clean["violation_name"].isin(top_names)].copy()

    name_pivot = pd.pivot_table(
        df_top,
        index=website_key,
        columns="violation_name",
        values="violation_count",
        aggfunc="sum",
        fill_value=0
    ).reset_index()

    # Optional: make column names ML-friendly (no spaces)
    name_pivot.columns = [c if c == website_key else f"vio_{c}".replace(" ", "_") for c in name_pivot.columns]
else:
    name_pivot = pd.DataFrame({website_key: df_clean[website_key].unique()})

display(name_pivot.head())
print("Top violation_name pivot shape:", name_pivot.shape)

,web_URL_id,vio_aria-allowed-attr,vio_aria-allowed-role,vio_aria-required-children,vio_aria-valid-attr-value,vio_button-name,vio_color-contrast,vio_color-contrast-enhanced,vio_duplicate-id,vio_duplicate-id-active,vio_duplicate-id-aria,vio_empty-heading,vio_frame-title,vio_heading-order,vio_html-has-lang,vio_image-alt,vio_image-redundant-alt,vio_landmark-one-main,vio_landmark-unique,vio_link-name,vio_list,vio_meta-viewport,vio_nested-interactive,vio_page-has-heading-one,vio_region,vio_scrollable-region-focusable
0,16,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,17,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0
2,20,0,0,0,0,0,0,3,3,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0
3,21,0,0,0,0,0,0,7,7,7,7,0,0,0,0,0,0,0,0,7,0,0,0,0,7,7
4,26,0,0,0,7,0,7,7,0,0,0,0,0,7,0,7,0,0,0,7,0,0,0,0,7,0


Top violation_name pivot shape: (558, 26)


In [ ]:
# Merge features
features = website_agg.merge(cat_pivot, on=website_key, how="left")
features = features.merge(name_pivot, on=website_key, how="left")

# Fill NaNs in pivot columns with 0
pivot_cols = [c for c in features.columns if c not in [website_key, "domain_category"] and features[c].dtype != "object"]
features[pivot_cols] = features[pivot_cols].fillna(0)

display(features.head())
print("Final features shape:", features.shape)

,web_URL_id,violation_count_sum,violation_score_mean,violation_score_median,violation_score_max,domain_category,layout,nan,semantic,syntax,vio_aria-allowed-attr,vio_aria-allowed-role,vio_aria-required-children,vio_aria-valid-attr-value,vio_button-name,vio_color-contrast,vio_color-contrast-enhanced,vio_duplicate-id,vio_duplicate-id-active,vio_duplicate-id-aria,vio_empty-heading,vio_frame-title,vio_heading-order,vio_html-has-lang,vio_image-alt,vio_image-redundant-alt,vio_landmark-one-main,vio_landmark-unique,vio_link-name,vio_list,vio_meta-viewport,vio_nested-interactive,vio_page-has-heading-one,vio_region,vio_scrollable-region-focusable
0,16,1,4.000000,4.0,4,government and public services,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17,16,3.250000,3.0,4,government and public services,4,0,0,12,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20,9,3.000000,3.0,4,government and public services,3,0,0,6,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21,49,3.571429,4.0,5,news and media,7,0,0,42,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,7.0
4,26,49,4.000000,4.0,5,news and media,14,0,0,35,0.0,0.0,0.0,7.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0


Final features shape: (591, 35)


In [ ]:
# Domain summary for charts
if "domain_category" in features.columns:
    domain_summary = features.groupby("domain_category").agg(
        n_websites=(website_key, "nunique"),
        total_violations=("violation_count_sum", "sum"),
        avg_violations_per_website=("violation_count_sum", "mean"),
    )

    # Add score summaries if available
    for score_col in ["violation_score_mean", "violation_score_median", "violation_score_max"]:
        if score_col in features.columns:
            domain_summary[f"avg_{score_col}"] = features.groupby("domain_category")[score_col].mean()

    domain_summary = domain_summary.reset_index()
    display(domain_summary.sort_values("avg_violations_per_website", ascending=False))
else:
    print("No domain_category available in features; skipping domain summary.")

,domain_category,n_websites,total_violations,avg_violations_per_website,avg_violation_score_mean,avg_violation_score_median,avg_violation_score_max
4,news and media,118,8568,72.610169,3.453289,3.487288,4.618644
0,e-commerce,55,2648,48.145455,3.515278,3.563636,4.400000
1,educational platforms,114,4740,41.578947,3.709690,3.741228,4.394737
5,streaming platforms,71,2838,39.971831,3.446464,3.478873,4.422535
6,technology science and research,112,3799,33.919643,3.632940,3.678571,4.419643
3,health and wellness,39,1123,28.794872,3.507326,3.461538,4.179487
2,government and public services,81,2310,28.518519,3.417726,3.333333,4.197531
7,technologyscienceresearch,1,3,3.000000,4.000000,4.000000,4.000000


In [ ]:
# Drop final nan cols
if "nan" in features.columns:
    features = features.drop(columns=["nan"])

In [ ]:
# Save cleaned + aggregated tables
df_clean.to_csv("clean_accessibility_data.csv", index=False)
features.to_csv("website_ml_features.csv", index=False)

if "domain_category" in features.columns:
    domain_summary.to_csv("domain_summary.csv", index=False)

print("Saved:")
print("- clean_accessibility_data.csv")
print("- website_ml_features.csv")
if "domain_category" in features.columns:
    print("- domain_summary.csv")

Saved:
- accessguru_clean.csv
- website_features.csv
- domain_summary.csv


In [ ]:
files.download("website_ml_features.csv")
files.download("clean_accessibility_data.csv")
if "domain_category" in features.columns:
    files.download("domain_summary.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>